In [1]:
import pandas as pd
import re
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import json
import warnings
warnings.filterwarnings("ignore")
import requests
import io

pd.set_option('display.max_columns', None)
df = pd.read_csv("Datasets/Dataset.csv")

df.head()

,Date,Tweet Id,Tweet Content,Tweet URL,Username,Tweet Language,Tweet Media,Tweet Coordinates,Tweet Place,Hastags,Cash Tags
0,2022-04-21 22:13:09+00:00,1.520000e+18,"#SonshineNewsblast: @doe_ph, nagdeklara ng pri...",https://twitter.com/DZAR1026/status/1517265208...,DZAR1026,tl,NaN,NaN,NaN,"['SonshineNewsblast', 'AgatonPH']",NaN
1,2022-04-21 21:10:35+00:00,1.520000e+18,Death toll sa Bagyong #AgatonPH lumobo na sa 2...,https://twitter.com/dzrhnews/status/1517249460...,dzrhnews,tl,NaN,NaN,NaN,"['AgatonPH', 'MagandangUmagaPilipinas', 'SamaS...",NaN
2,2022-04-21 20:01:13+00:00,1.520000e+18,#ICYMI: Angel Locsin explained that she chose ...,https://twitter.com/inquirerdotnet/status/1517...,inquirerdotnet,en,NaN,NaN,NaN,"['ICYMI', 'AgatonPH']",NaN
3,2022-04-21 19:15:00+00:00,1.520000e+18,BINAWI ng National Disaster Risk Reduction and...,https://twitter.com/AbanteNews/status/15172203...,AbanteNews,tl,NaN,NaN,NaN,"['TunayNaTabloidista', 'AbanteNews', 'AgatonPH']",NaN
4,2022-04-21 16:48:11+00:00,1.520000e+18,تحديث | أفاد المجلس الوطني للحد من مخاطر الكوا...,https://twitter.com/Philippine_live/status/151...,Philippine_live,ar,[Photo(previewUrl='https://pbs.twimg.com/media...,NaN,NaN,['AgatonPH'],NaN


In [2]:
# Set Language here
LANGUAGE = "en"
if LANGUAGE == "tl":
    # Join english stopwords list and tagalog stopwords list
    tagalog_stops = pd.read_csv("Datasets/tagalog_stopwords.csv").a.to_list()
    stop_words = stopwords.words('english') + tagalog_stops
else:
    stop_words = stopwords.words('english')
temp = df[(df["Tweet Language"] == LANGUAGE)]

In [3]:
# Clean data using regex and by tokenizing
def preprocessing_text(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+https\S+", '',text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','',text)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

tweet_df = pd.DataFrame()
# Apply cleaning
tweet_df['cleaned'] = temp['Tweet Content'].apply(preprocessing_text)
tweet_df['coords'] = temp['Tweet Coordinates']
tweet_df = tweet_df.reset_index(drop=True)
tweet_df

,cleaned,coords
0,icymi angel locsin explained chose wear pink r...,NaN
1,congrats 3rd leg donation drive alyfinity2 nex...,NaN
2,another batch relief distributed landslide sur...,NaN
3,recorded deaths eastern visayas hundred people...,NaN
4,donationsagatonph nbaplayoffs givetheearthaglo...,NaN
...,...,...
8256,bicolpossible rains whole week due preagatonph...,NaN
8257,initial guide tourist spots holy week boracay ...,NaN
8258,name malakas contributed philippines means str...,NaN
8259,ecmwf projections show lpa forming east mindan...,NaN


In [9]:
json_city = json.load(open("Datasets/cities.geojson", "r"))
id_map = {}
city_map = {}
province_map = {}
index = 0
for feature in json_city["features"]:
    #feature["id"] = feature["properties"]["ID_2"]
    feature["id"] = index
    id_map[feature["properties"]["NAME_1"].lower() + " " + feature["properties"]["NAME_2"].lower()] = feature["id"]
    city_map[feature["properties"]["NAME_2"].lower()] = feature["id"]
    province_map[feature["properties"]["NAME_1"].lower()] = feature["id"]
    index += 1
print(len(id_map))
print(len(city_map))
print(len(province_map))
province_map

1647
1425
82


{'abra': 124,
 'agusan del norte': 20,
 'isabela': 784,
 'kalinga': 33,
 'la union': 795,
 'shariff kabunsuan': 1444,
 'siquijor': 52,
 'sorsogon': 1448,
 'zamboanga del sur': 1644,
 'zamboanga sibugay': 1646,
 'camiguin': 468,
 'capiz': 104,
 'catanduanes': 478,
 'agusan del sur': 138,
 'aklan': 155,
 'albay': 174,
 'antique': 192,
 'apayao': 199,
 'aurora': 207,
 'basilan': 218,
 'bataan': 230,
 'batanes': 236,
 'batangas': 271,
 'benguet': 285,
 'biliran': 293,
 'bohol': 341,
 'bukidnon': 363,
 'bulacan': 387,
 'cagayan': 416,
 'camarines norte': 428,
 'camarines sur': 467,
 'cavite': 501,
 'cebu': 555,
 'compostela valley': 566,
 'davao del norte': 577,
 'davao del sur': 593,
 'davao oriental': 604,
 'dinagat islands': 611,
 'eastern samar': 634,
 'guimaras': 639,
 'ifugao': 650,
 'ilocos norte': 674,
 'ilocos sur': 708,
 'iloilo': 752,
 'laguna': 830,
 'lanao del norte': 853,
 'lanao del sur': 895,
 'leyte': 959,
 'maguindanao': 974,
 'marinduque': 947,
 'masbate': 989,
 'metropol

In [5]:
no_coords = tweet_df[tweet_df["coords"].isna()]
with_coords = tweet_df.dropna(subset="coords")

In [6]:
def locate_tweet(text):
    result = ""
    text_tokens = word_tokenize(text)
    city = ""
    province = ""
    init_loc = ""
    for w in text_tokens:
        if city_map.__contains__(w) and city == "":
            city = w
            init_loc = w + " " + init_loc
        if province_map.__contains__(w) and province == "":
            province = w
            init_loc = init_loc  + w
        if city != "" and province != "":
            init_loc.strip()
            break;
    index_loc = 0
    if id_map.__contains__(init_loc):
        index_loc = id_map[init_loc]
    elif city != "":
        index_loc = city_map[city]
    elif province != "":
        index_loc = province_map[province]
    else:
        return np.NaN
    loc = json_city["features"][int(index_loc)]["geometry"]["coordinates"][0][0]
    location = re.findall('\\d+.\\d+', str(loc))

    return str(location[1])+"/"+str(location[0])+":"+init_loc

no_coords['coords'] = no_coords['cleaned'].apply(locate_tweet)
no_coords = no_coords.dropna(subset="coords")

In [7]:
i = 0
while True:
    loc = json_city["features"][830- i]["geometry"]
    if loc != None:
        loc = json_city["features"][830- i]["geometry"]["coordinates"][0][0]
        break
    i =+ 1
loc

[121.370506, 14.19444]

In [8]:
def add_location_name(text):
    return text.split(":")[1]

no_coords['loc_name'] = no_coords['coords'].apply(add_location_name)
no_coords

,cleaned,coords,loc_name
0,icymi angel locsin explained chose wear pink r...,11.33877/124.552177:leyte leyte,leyte leyte
2,another batch relief distributed landslide sur...,11.33877/124.552177:leyte leyte,leyte leyte
8,actressphilanthropist angel locsin explained c...,11.33877/124.552177:leyte leyte,leyte leyte
10,angel locsin explained chose wear pink relief ...,11.33877/124.552177:leyte leyte,leyte leyte
12,usaidsaveslives us provided relief hygiene kit...,11.33877/124.552177:leyte leyte,leyte leyte
...,...,...,...
8245,3 saturday center tropical depression agatonph...,10.77604/125.957024:guiuan samar,guiuan samar
8247,breaking lpa east guiuan eastern samar intensi...,10.77604/125.957024:guiuan samar,guiuan samar
8253,breaking pagasa says low pressure area lpa cur...,10.77604/125.957024:guiuan samar,guiuan samar
8254,severe weather warning please share areas seve...,9.96/125.61573:dinagat samar,dinagat samar


In [9]:
def preprocess_coords(coords):
    numbers = re.findall('\\d+', coords)
    return numbers[2]+"."+numbers[3]+"/"+numbers[0]+"."+numbers[1]

with_coords['coords'] = with_coords['coords'].apply(preprocess_coords)
no_coords['coords'] = no_coords['coords'].apply(preprocess_coords)
print(no_coords.shape[0])
print(with_coords.shape[0])
merge = pd.concat([no_coords, with_coords], axis=0).reset_index(drop=True)
merge.to_csv('Datasets/' + 'dataset_with_coords.csv', index = False)

2478
126
